In [15]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
%matplotlib inline 

In [16]:
dataset = "Path to the dataset edges file"
embedding_filename = "Path where you want to save the embedding file"
walk_length = 100
num_walks = 80
embed_size=50
iter = 1
window_size=10



In [17]:
df = pd.read_csv(dataset,
                sep = '\t',
                names = ["NodeIDfrom", "NodeIDto"],
                skiprows=1)
df

,NodeIDfrom,NodeIDto
0,1,0
1,2,3
2,0,28
3,3,2
4,4,5
...,...,...
62358,9431,1153
62359,9432,4234
62360,9433,4234
62361,9434,4234


In [18]:
#create the graph networkx object from the above dataframe

G = nx.from_pandas_edgelist(df = df,
                             source = "NodeIDfrom",
                             target = "NodeIDto",
                             create_using=nx.Graph())
len(G)

9436

In [19]:
def partition_num(num, workers):
    if num % workers == 0:
        return [num // workers] * workers
    else:
        return [num // workers] * workers + [num % workers]

In [20]:
import numpy as np


def create_alias_table(area_ratio):
    """

    :param area_ratio: sum(area_ratio)=1
    :return: accept,alias
    """
    l = len(area_ratio)
    accept, alias = [0] * l, [0] * l
    small, large = [], []
    area_ratio_ = np.array(area_ratio) * l
    for i, prob in enumerate(area_ratio_):
        if prob < 1.0:
            small.append(i)
        else:
            large.append(i)

    while small and large:
        small_idx, large_idx = small.pop(), large.pop()
        accept[small_idx] = area_ratio_[small_idx]
        alias[small_idx] = large_idx
        area_ratio_[large_idx] = area_ratio_[large_idx] - \
                                 (1 - area_ratio_[small_idx])
        if area_ratio_[large_idx] < 1.0:
            small.append(large_idx)
        else:
            large.append(large_idx)

    while large:
        large_idx = large.pop()
        accept[large_idx] = 1
    while small:
        small_idx = small.pop()
        accept[small_idx] = 1

    return accept, alias


def alias_sample(accept, alias):
    """

    :param accept:
    :param alias:
    :return: sample index
    """
    N = len(accept)
    i = int(np.random.random() * N)
    r = np.random.random()
    if r < accept[i]:
        return i
    else:
        return alias[i]


In [21]:
import itertools
import math
import random

import pandas as pd
from joblib import Parallel, delayed


class RandomWalker:
    def __init__(self, G, use_rejection_sampling=False):
        """
        :param G: networkx Graph Object.
        
        """
        self.G = G
        self.use_rejection_sampling = use_rejection_sampling

    def node2vec_walk(self, walk_length, start_node):

        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = list(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(
                        cur_nbrs[alias_sample(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    edge = (prev, cur)
                    next_node = cur_nbrs[alias_sample(alias_edges[edge][0],
                                                      alias_edges[edge][1])]
                    walk.append(next_node)
            else:
                break

        return walk
    
    def simulate_walks(self, num_walks, walk_length, workers=1, verbose=0):

        G = self.G

        nodes = list(G.nodes())

        results = Parallel(n_jobs=workers, verbose=verbose, )(
            delayed(self._simulate_walks)(nodes, num, walk_length) for num in
            partition_num(num_walks, workers))

        walks = list(itertools.chain(*results))

        return walks

    def _simulate_walks(self, nodes, num_walks, walk_length, ):
        walks = []
        for _ in range(num_walks):
            random.shuffle(nodes)
            for v in nodes:
                walks.append(self.node2vec_walk(
                    walk_length=walk_length, start_node=v))
        return walks
    
    def preprocess_transition_probs(self):
        """
        Preprocessing of transition probabilities for guiding the random walks.
        """
        G = self.G
        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr].get('weight', 1.0)
                                  for nbr in G.neighbors(node)]
            norm_const = sum(unnormalized_probs)
            normalized_probs = [
                float(u_prob) / norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = create_alias_table(normalized_probs)

        if not self.use_rejection_sampling:
            alias_edges = {}

            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
                if not G.is_directed():
                    alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])
                self.alias_edges = alias_edges

        self.alias_nodes = alias_nodes
        return
    
    def get_alias_edge(self, t, v):
        """
        compute unnormalized transition probability between nodes v and its neighbors give the previous visited node t.
        :param t:
        :param v:
        :return:
        """
        G = self.G
        p = 1
        q = 1

        unnormalized_probs = []
        for x in G.neighbors(v):
            weight = G[v][x].get('weight', 1.0)  # w_vx
            if x == t:  # d_tx == 0
                unnormalized_probs.append(weight / p)
            elif G.has_edge(x, t):  # d_tx == 1
                unnormalized_probs.append(weight)
            else:  # d_tx > 1
                unnormalized_probs.append(weight / q)
        norm_const = sum(unnormalized_probs)
        normalized_probs = [
            float(u_prob) / norm_const for u_prob in unnormalized_probs]

        return create_alias_table(normalized_probs)
    
    

In [22]:
from gensim.models import Word2Vec
class Node2Vec:

    def __init__(self, graph, walk_length, num_walks, workers=1, use_rejection_sampling = False):

        self.graph = graph
        self.walker = RandomWalker(G)

        print("Preprocess transition probs...")
        self.walker.preprocess_transition_probs()

        self.sentences = self.walker.simulate_walks(
            num_walks=num_walks, walk_length=walk_length, workers=workers, verbose=1)

    def train(self, embed_size=128, window_size=5, workers=3, iter=5, **kwargs):
        kwargs["sentences"] = self.sentences
        kwargs["min_count"] = kwargs.get("min_count", 0)
        kwargs["vector_size"] = embed_size
        kwargs["sg"] = 1
        kwargs["hs"] = 0  # node2vec not use Hierarchical Softmax
        kwargs["workers"] = workers
        kwargs["window"] = window_size
        kwargs["epochs"] = iter

        print("Learning embedding vectors...")
        model = Word2Vec(**kwargs)
        print("Learning embedding vectors done!")

        self.w2v_model = model

        return model

In [23]:
model = Node2Vec(graph = G,
                 walk_length = 100,
                 num_walks = 80)
model.train(embed_size=50,
            iter = 1,
            window_size=10)

Preprocess transition probs...
Learning embedding vectors...
Learning embedding vectors done!


In [24]:
model.w2v_model.wv[0]

array([-0.11270191,  0.49906412,  0.50348854, -0.34926018,  0.2177425 ,
       -0.12523465,  0.8685004 ,  0.3002656 ,  0.7236134 , -0.14671412,
       -0.31611195, -0.11450509,  0.42640367,  0.15217224,  0.50814617,
        0.04632494,  0.17245506, -0.33699244, -0.7271709 ,  0.61405146,
       -0.01804817,  0.09432548, -0.33093464,  0.38034645, -0.10888965,
        0.3004258 ,  0.27281588,  0.28487545,  0.00183553,  0.43650576,
        0.49253207, -0.18696032,  0.0210338 , -0.09092064,  0.42407435,
       -0.4031546 , -0.57301676,  0.05752803, -0.64204496, -0.4661547 ,
       -0.82910365, -0.20246585,  0.78567225,  0.05741467,  0.20295218,
        0.27117878,  0.11639062, -0.48698613,  0.64499944, -0.45387548],
      dtype=float32)

In [25]:
embedding_filename = "/Users/vanshgupta/Desktop/AI and ML reading material/GraphGAN_Project/Emb and Data/Emb/biogrid-human/Node2Vec/emb.txt"

In [26]:
embeddings = []
for i in G.nodes():
    embeddings.append(model.w2v_model.wv[i]) #/np.linalg.norm(model.wv[i]))
embeddings = np.array(embeddings)
embeddings[0], np.max(embeddings)

(array([-0.4411929 , -0.23272078,  0.34105706, -0.12732305,  0.14284867,
        -0.05249456,  1.3319893 , -0.07911512,  0.5341532 ,  0.18609825,
        -0.0388088 ,  0.37535933,  0.07543252,  0.21904887, -0.22800668,
        -0.60518056, -0.7063029 , -0.6923662 , -0.770631  ,  0.6574531 ,
         0.2216889 , -0.4124251 ,  0.14412065,  0.05070386, -0.05478349,
         0.06379271,  0.7106666 ,  0.20269437,  0.63162607,  0.1648631 ,
         0.74775845, -0.9446173 , -0.45626616, -0.23624764,  0.64157426,
         0.06679013, -0.621122  , -0.05883688, -0.0942089 , -0.5949024 ,
        -0.63340694, -0.3138731 , -0.13964635,  0.10933653, -0.21421202,
         0.07387857,  0.3730023 , -0.35212922,  0.32944885, -0.34283614],
       dtype=float32),
 3.5945706)

In [27]:
import os
index = np.array(G.nodes()).reshape(-1, 1)
embedding_matrix = np.hstack([index, embeddings])
embedding_list = embedding_matrix.tolist()
embedding_str = [str(int(emb[0])) + " " + " ".join([str(x) for x in emb[1:]]) + "\n"
                  for emb in embedding_list]
with open(embedding_filename, "w+") as f:
    lines = [str(G.number_of_nodes()) + "\t" + str(50) + "\n"] + embedding_str
    f.writelines(lines)